In [ ]:
import torch
import torch.nn.functional as F
from resnet18 import *
from char2img import *
from letters import *

if torch.cuda.is_available():
    print("using Cuda")
    device = torch.device("cuda")
elif torch.backends.mps.is_built():
    print("using MPS")
    device = torch.device("mps")
else:
    print("using CPU")
    device = torch.device("cpu")

In [ ]:
# Load the model
experiment_name = "robust"
#model = ResNet18(num_classes=ABC_SIZE + len(additional_symbols))
model = ResNet18()
model.load_state_dict(torch.load(f'models/resnet18_{experiment_name}.pth'))
model = model.to(device)

# Compute the mean and std of the dataset
dataset = LettersDataset(f"data/letters_{experiment_name}.csv", device)
mean = torch.mean(torch.stack([array.mean() for array, _ in dataset]))
std = torch.std(torch.stack([array.std() for array, _ in dataset]))
print(f"{mean = }")
print(f"{std = }")

In [ ]:
# Perform inference
def inference(text, model):
    array = generate_chars_image(text)
    show_img(array)
    tensor = torch.tensor(array).unsqueeze(0).unsqueeze(0).float().to(device)
    tensor = (tensor - mean) / std
    model.eval()
    with torch.no_grad():
        # Inference
        logits = model(tensor)

        # Extract only the logits corresponding to letters
        letter_logits = logits[:, :ABC_SIZE]
        
        # Calculate softmax probabilities only on letter logits
        probas = F.softmax(letter_logits, dim=1)

        # Print the predicted label
        predicted_label = torch.argmax(probas).item()
        predicted_prob = probas[0][predicted_label]
        print(f"Label: {chr(predicted_label + ord('A'))}, Probability: {predicted_prob:.4f}")

        # Print other labels with relatively high probability
        for i in range(len(probas[0])):
            if probas[0][i] > (predicted_prob * 0.5) and i != predicted_label:
                print(f"Label: {chr(i + ord('A'))}, Probability: {probas[0][i]:.4f}")

In [ ]:
def evaluate(text_list, labels_list, model):
    dataset_size = len(text_list)
    num_correct = 0
    model.eval()
    with torch.no_grad():
        for text, label in zip(text_list, labels_list):
            array = generate_chars_image(text)

            # Normalize
            tensor = torch.tensor(array).unsqueeze(0).unsqueeze(0).float().to(device)
            tensor = (tensor - mean) / std

            # Inference
            logits = model(tensor)

            # Extract only the logits corresponding to letters
            letter_logits = logits[:, :ABC_SIZE]
            
            # Calculate softmax probabilities only on letter logits
            probas = F.softmax(letter_logits, dim=1)
            
            # Get the predicted letter class
            predicted_class = chr(torch.argmax(probas).item() + ord('A'))
            
            if predicted_class == label:
                num_correct += 1

    return round((num_correct / dataset_size) * 100, 2)

In [ ]:
# evaluate upper case letters are predicted correctly
accuracy = evaluate(upper_case_alphabet, upper_case_alphabet, model)
print(f"Upper-case accuracy: {accuracy}")

# evaluate upper case letters are predicted correctly
accuracy = evaluate(lower_case_alphabet, upper_case_alphabet, model)
print(f"Lower-case accuracy: {accuracy}")

In [ ]:
# evaluate leet speak substitutions are predicted correctly

total_accuracy = 0
for letter in upper_case_alphabet:
    n_subs = len(leet_speak_alphabet[letter])
    labels = [letter] * n_subs
    accuracy = evaluate(leet_speak_alphabet[letter], labels, model)
    print(f"Letter {letter} accuracy: {accuracy}")
    total_accuracy += accuracy
print(f"Total accuracy: {total_accuracy / len(upper_case_alphabet)}")

In [ ]:
for letter in lower_case_alphabet:
    inference(letter, model)

In [ ]:
inference('|-|', model)

In [ ]:
for letter in upper_case_alphabet:
    for sub in leet_speak_alphabet[letter]:
        inference(sub, model)